# Imports

In [1]:
# === Imports (centralized) ===
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge
import boto3
from pyathena import connect

# Tables Uploads

In [2]:

import os


In [3]:
# UPLOADING Heatmap
conn = connect(s3_staging_dir=ATHENA_RESULTS_S3, region_name=REGION, work_group="primary")
sql = "SELECT * FROM little_caesars.heatmap"
heatmap = pd.read_sql(sql, conn)
heatmap.head()

C:\Users\NayyeraAboelnaga\AppData\Local\Temp\ipykernel_13352\69582114.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  heatmap = pd.read_sql(sql, conn)


,store_number,carryout_sales_pizza,carryout_sales_non_cash_coupon,delivery_sales_pizza_mobile_app,carryout_sales_school_lunch_or_church_org,carryout_sales_promotional_sales,carryout_sales_sandwich,carryout_sales_salad,carryout_sales_dessert,carryout_sales_alcoholic_beverage,...,ingestion_date,organizational_expenses,rou_lease_expense,business_tax_expense,controllable_profits,profit_sales_pct,store_lc,year,month,day
0,364001,20057.84,0,8801.87,0,0,0,0,0,0,...,09/18/2025,nan,nan,nan,None,None,None,2022,09,05
1,364002,0.0,0,0.0,0,0,0,0,0,0,...,09/18/2025,nan,nan,nan,None,None,None,2022,09,05
2,364003,30022.03,0,4999.8,0,0,0,0,0,0,...,09/18/2025,nan,nan,nan,None,None,None,2022,09,05
3,364004,26107.15,0,4947.94,0,0,0,0,0,0,...,09/18/2025,nan,nan,nan,None,None,None,2022,09,05
4,364005,28626.25,0,3325.07,0,0,0,0,0,0,...,09/18/2025,nan,nan,nan,None,None,None,2022,09,05


In [4]:
# UPLOADING Hierachy
conn = connect(s3_staging_dir=ATHENA_RESULTS_S3, region_name=REGION, work_group="primary")
sql = "SELECT * FROM little_caesars.hierarchy"
geo_stores = pd.read_sql(sql, conn)
geo_stores.columns = geo_stores.iloc[0]
geo_stores = geo_stores[1:].reset_index(drop=True)
geo_stores.head()

C:\Users\NayyeraAboelnaga\AppData\Local\Temp\ipykernel_13352\1102283296.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  geo_stores = pd.read_sql(sql, conn)


,Store_Number,Heatmap_Store_Number,Reduced_Store_Number,Store_Name,Full_Store,Director,DO,Market,Store_Address,State,Director_Email,Email_Address,Date,None,2025
0,03647-00011,364011,11,KCK,KCK 03647-00011,,Josh Butler,KS Market,151 S. 18th Street Suite K,Kansas,,jbutler@apricusqsr.com,2023-10-22,None,2025
1,03647-00012,364012,12,Leavenworth,Leavenworth 03647-00012,,Josh Butler,KS Market,2024 S. 4th Street Leavenworth,Kansas,,jbutler@apricusqsr.com,2023-10-22,None,2025
2,03647-00013,364013,13,Topeka,Topeka 03647-00013,,Josh Butler,KS Market,5329 SW 21st Street Topeka,Kansas,,jbutler@apricusqsr.com,2023-10-22,None,2025
3,03647-00014,364014,14,Olathe,Olathe 03647-00014,,Josh Butler,KS Market,904 E Santa Fe St. Olathe,Kansas,,jbutler@apricusqsr.com,2023-10-22,None,2025
4,03647-00015,364015,15,Lawrence,Lawrence 03647-00015,,Josh Butler,KS Market,1528 W 23rd St. Lawrence,Kansas,,jbutler@apricusqsr.com,2023-10-22,None,2025


In [5]:
# Calendar Dates Flagged
events = pd.read_excel('data/events_flag.xlsx')
#1) Load fiscal calendar ---
cal = pd.read_excel('data/fiscal_calendar (1).xlsx')
cpi_raw =  pd.read_excel('data/cpi_data.xlsx')

In [6]:
# stores to excludeb agreed with Leonor
exclude_stores = [364001, 364002, 364003, 364004, 364005, 364006, 364007, 364008, 364009, 364010]

# Select required columns and exclude the stores
h = heatmap[
    ~heatmap["store_number"].isin(exclude_stores)
][[
    "event_date",
    "total_net_sales",
    "store_number",
    "total_advertising"
]]

h.head()


,event_date,total_net_sales,store_number,total_advertising
0,2022-09-05,44227.86,364001,1946.03
1,2022-09-05,0.0,364002,0.0
2,2022-09-05,51766.14,364003,2277.71
3,2022-09-05,47644.26000000001,364004,2096.35
4,2022-09-05,47142.11,364005,2074.25


#final

In [7]:
import sys, os, importlib, pandas as pd, numpy as np
sys.path.append(os.getcwd())
import forecasting.pooled_ridge as pr
importlib.reload(pr)
from forecasting.pooled_ridge import ForecastConfig, FeatureBuilder, PooledRidgeForecaster

# ========= Helpers =========
def std_fiscal_cols(cal: pd.DataFrame) -> pd.DataFrame:
    c = cal.copy()
    lower = {col.lower(): col for col in c.columns}
    fy = lower.get('fiscal_year') or lower.get('year') or lower.get('fy') or lower.get('fiscalyear')
    fp = lower.get('fiscal_period') or lower.get('period') or lower.get('fp') or lower.get('fiscalperiod')
    if fy is None or fp is None:
        raise ValueError("Calendar must include fiscal year/period columns.")
    if fy != 'fiscal_year' or fp != 'fiscal_period':
        c = c.rename(columns={fy: 'fiscal_year', fp: 'fiscal_period'})
    for col in ['BeginningDate','EndingDate']:
        if col in c.columns:
            c[col] = pd.to_datetime(c[col], errors='coerce').dt.normalize()
    return c

def build_events_df_fiscal(events_path: str, cal2: pd.DataFrame) -> pd.DataFrame:
    ev = pd.read_excel(events_path)
    ev['event_date'] = pd.to_datetime(ev['event_date'], errors='coerce').dt.normalize()
    sb_col = None
    for c in ev.columns:
        k = c.strip().lower().replace(' ','')
        if k in ('superbowl','super_bowl'):
            sb_col = c; break
    if sb_col is None:
        raise ValueError("No 'Super Bowl' column in events file.")
    ev = ev[['event_date', sb_col]].rename(columns={sb_col: 'Super Bowl'})
    ev['Super Bowl'] = pd.to_numeric(ev['Super Bowl'], errors='coerce').fillna(0).astype(int).clip(0,1)

    span = cal2[['BeginningDate','EndingDate','fiscal_year','fiscal_period']].dropna().copy()
    span['key'] = 1; ev['key'] = 1
    m = (ev.merge(span, on='key', how='inner')
            .query('BeginningDate <= event_date <= EndingDate'))
    events_df = (m[['fiscal_year','fiscal_period','BeginningDate','Super Bowl']]
                 .drop_duplicates(['fiscal_year','fiscal_period'])
                 .rename(columns={'BeginningDate':'event_date'}))
    return events_df[['event_date','fiscal_year','fiscal_period','Super Bowl']].drop_duplicates()

def sanitize_training_features(df: pd.DataFrame, feature_cols: list, target_col: str) -> pd.DataFrame:
    out = df.copy()
    out['store_number'] = out['store_number'].astype(str).str.strip()
    for c in feature_cols:
        out[c] = pd.to_numeric(out[c], errors='coerce')
    out[feature_cols] = out[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    out[target_col] = pd.to_numeric(out[target_col], errors='coerce')
    out = out.dropna(subset=[target_col])
    return out

def forecast_baseline_H(history_df, cal_df, fb, forecaster, events_df, H=6):
    out = []
    hist = history_df.copy()
    store_col = forecaster.cfg.store_col
    feat_cols = forecaster.cfg.feature_list
    cal2 = std_fiscal_cols(cal_df)

    for step in range(1, H+1):
        fut = fb.build_forecast_set(history_df=hist, calendar_df=cal2, events_df=events_df)
        fut[store_col] = fut[store_col].astype(str).str.strip()
        for c in feat_cols:
            fut[c] = pd.to_numeric(fut[c], errors='coerce').replace([np.inf,-np.inf], np.nan).fillna(0.0)

        pred = forecaster.predict(fut).assign(h=step)
        pred = pred.merge(cal2[['fiscal_year','fiscal_period','BeginningDate']].drop_duplicates(),
                          on=['fiscal_year','fiscal_period'], how='left') \
                   .rename(columns={'BeginningDate':'event_date'})
        out.append(pred)

        fut_upd = fut.copy()
        fut_upd['total_net_sales'] = pred['forecast'].values
        fut_upd['event_date'] = pd.to_datetime(fut_upd['event_date'], errors='coerce')
        hist = pd.concat([hist, fut_upd], ignore_index=True).sort_values([store_col,'event_date'])

    return pd.concat(out, ignore_index=True)

# ========= Config =========
EXCLUDE = [364001,364002,364003,364004,364005,364006,364007,364008,364009,364010]
FEATURES = ['total_net_sales_m1','adv_m1','CPI_m1','p13_sin','p13_cos','is_5w','Super Bowl']

cfg = ForecastConfig(
    feature_list=FEATURES,
    target='total_net_sales',
    store_col='store_number',
    time_cols=('event_date','fiscal_year','fiscal_period'),
    alpha=5.0,
    exclude_stores=EXCLUDE
)
fb = FeatureBuilder(cfg)

# ========= Build features (train set) with fiscal-aligned events =========
cal2 = std_fiscal_cols(cal)
events_df = build_events_df_fiscal('data/events_flag.xlsx', cal2)

feats = fb.build_feature_table(
    base_df=h,                # your heatmap subset with required columns
    calendar_df=cal2,
    events=events_df,         # pass DataFrame, not path
    cpi='data/cpi_data.xlsx'
)

# ========= Sanitize features (fix NaNs/inf) =========
feats = sanitize_training_features(feats, FEATURES, cfg.target)

# ========= Train =========
forecaster = PooledRidgeForecaster(cfg)
_ = forecaster.train(feats)

# ========= Forecast per store (H steps) =========
H = 6
baseline_table = forecast_baseline_H(
    history_df=feats,         # use feature history for stable lags
    cal_df=cal2,
    fb=fb,
    forecaster=forecaster,
    events_df=events_df,
    H=H
)
baseline_table = baseline_table.drop(columns=['event_date'], errors='ignore')


# Final table (no export) — sorted for readability
baseline_table = (baseline_table
                  .sort_values(['store_number','fiscal_year','fiscal_period','h'])
                  .reset_index(drop=True))

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', 0,
                       'display.float_format', '{:,.2f}'.format):
    display(baseline_table)


[FeatureBuilder] excluded stores: 10 | rows removed: 116
[FeatureBuilder] final features rows: 1292 (dropped 61)
[Train] holdout WAPE: 0.0547 | rows_eval=168
[Train] stores_train=31 | stores_eval=28 | rows_train=859 | rows_eval=168 | alpha=5.0
[FeatureBuilder] next-period design rows: 61
[FeatureBuilder] next-period design rows: 61
[FeatureBuilder] next-period design rows: 61
[FeatureBuilder] next-period design rows: 61
[FeatureBuilder] next-period design rows: 61
[FeatureBuilder] next-period design rows: 61


,store_number,fiscal_year,fiscal_period,forecast,h
0,364011,2025,10,"163,954.03",1
1,364011,2025,11,"163,937.60",2
2,364011,2025,12,"164,526.58",3
3,364011,2025,13,"165,586.04",4
4,364011,2026,1,"166,873.27",5
5,364011,2026,2,"168,093.38",6
6,364012,2025,10,"60,405.07",1
7,364012,2025,11,"60,388.64",2
8,364012,2025,12,"60,977.62",3
9,364012,2025,13,"62,037.08",4
